## Distancia de Mahanalobis

In [1]:
import time
import numpy as np
import pandas as pd
from itertools import combinations
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, max_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from scipy.stats import chi2
from scipy.spatial.distance import cdist

# Función para calcular la distancia de Mahalanobis de cada punto
def mahalanobis_distance(x, data):
    data = data.astype(float)
    covariance_matrix = np.cov(data, rowvar=False)
    inv_covariance_matrix = np.linalg.inv(covariance_matrix)
    diff = x - np.mean(data, axis=0)
    md = np.sqrt(np.dot(np.dot(diff, inv_covariance_matrix), diff.T))
    return md

# Cargar datos desde el archivo CSV
df_datos = pd.read_csv("datos/insurance.csv")

# Codificación One Hot para las columnas 'sex' y 'region'
df_datos['sex'] = df_datos['sex'].map({'male': 0, 'female': 1})
codificador_one_hot = OneHotEncoder(sparse=False)
codificacion_one_hot = codificador_one_hot.fit_transform(df_datos[['region']])
arr_nombre_nuevas_columnas = ['region_' + str(cat) for cat in codificador_one_hot.categories_[0]]
df_nuevas_columnas_one_hot = pd.DataFrame(codificacion_one_hot, columns=arr_nombre_nuevas_columnas)
df_datos = pd.concat([df_datos, df_nuevas_columnas_one_hot], axis=1)

# Columnas a utilizar incluyendo 'region'
columnas_especificas = ['age', 'bmi'] + arr_nombre_nuevas_columnas + ['children', 'sex']

# Crear conjuntos de datos de entrenamiento y prueba
df_X = pd.DataFrame(df_datos, columns=columnas_especificas)
df_y = pd.DataFrame(df_datos, columns=['charges'])
df_X_train, df_X_test, df_y_train, df_y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=100)

# Medir tiempo de preprocesamiento
start_time_preprocesamiento = time.time()

# Calcular el punto medio de los datos
punto_medio = df_X_train.mean()

# Calcular la matriz de covarianza de los datos de entrenamiento
cov_matrix = np.cov(df_X_train.values, rowvar=False)

# Calcular la matriz de distancias utilizando cdist
distancias = cdist(df_X_train.values, [punto_medio.values], 'mahalanobis', VI=cov_matrix)

# Convertir las distancias a un array unidimensional
distancias_unidimensional = distancias.flatten()

# Calcular el umbral de chi-cuadrado
alpha = 0.05
chi2_threshold = chi2.ppf(1 - alpha, df=df_X_train.shape[1])

# Encontrar los índices de los datos anómalos
anomaly_indices = np.argsort(-distancias_unidimensional)[:int(alpha * len(distancias_unidimensional))]

# Eliminar los datos anómalos del conjunto de entrenamiento y sus etiquetas correspondientes
df_X_train_clean = df_X_train.drop(df_X_train.index[anomaly_indices])
df_y_train_clean = df_y_train.drop(df_y_train.index[anomaly_indices])

# Tiempo de eliminación de anomalías
tiempo_elim_anomalias = time.time() - start_time_preprocesamiento

# Entrenar modelos de regresión polinómica para diferentes grados
resultados_polinomicos = []
grados_polinomicos = [2, 4, 6]

for grado in grados_polinomicos:
    for r in range(1, len(columnas_especificas) + 1):
        combinaciones_columnas = combinations(columnas_especificas, r)
        for combo in combinaciones_columnas:
            arr_X_train_subset = df_X_train_clean[list(combo)].copy()
            arr_X_test_subset = df_X_test[list(combo)].copy()
            arr_X_train_subset.fillna(arr_X_train_subset.mean(), inplace=True)
            arr_X_test_subset.fillna(arr_X_test_subset.mean(), inplace=True)
            if len(arr_X_train_subset) == 0 or len(arr_X_test_subset) == 0:
                continue
            start_time_modelo = time.time()
            model = make_pipeline(PolynomialFeatures(degree=grado), LinearRegression())
            model.fit(arr_X_train_subset, df_y_train_clean)
            tiempo_modelo = time.time() - start_time_modelo
            start_time_prueba = time.time()
            arr_y_test_predicho = model.predict(arr_X_test_subset)
            max_ea = max_error(df_y_test, arr_y_test_predicho)
            mae = mean_absolute_error(df_y_test, arr_y_test_predicho)
            mse = mean_squared_error(df_y_test, arr_y_test_predicho)
            r2 = r2_score(df_y_test, arr_y_test_predicho)
            tiempo_prueba = time.time() - start_time_prueba
            resultados_polinomicos.append({
                'Grado_Polinómico': grado,
                'Combinación_Columnas': ', '.join(combo),
                'Máximo_EA': max_ea,
                'EA_Medio': mae,
                'R^2': r2,
                'Tiempo_Modelo': tiempo_modelo,
                'Tiempo_Prueba': tiempo_prueba
            })

# Crear un DataFrame con los resultados
df_resultados_polinomicos = pd.DataFrame(resultados_polinomicos)

# Imprimir el tiempo total de preprocesamiento
tiempo_preprocesamiento_total = time.time() - start_time_preprocesamiento + tiempo_elim_anomalias
print(f"Tiempo de preprocesamiento total: {tiempo_preprocesamiento_total} segundos")
print('***************************************************************************************')

# Ordenar el DataFrame por índice en orden ascendente
df_resultados_polinomicos = df_resultados_polinomicos.sort_index()

# Imprimir resultados
for idx, row in df_resultados_polinomicos.iterrows():
    print(f"{idx + 1:2d}. Grado Polinómico: {row['Grado_Polinómico']}, Combinación: {row['Combinación_Columnas']}")
    print(f"   Máximo EA: {row['Máximo_EA']:.2f}")
    print(f"   EA Medio: {row['EA_Medio']:.2f}")
    print(f"   R^2: {row['R^2']:.4f}")
    print(f"   Tiempo de Generación del Modelo: {row['Tiempo_Modelo']:.4f} segundos")
    print(f"   Tiempo de Prueba del Modelo: {row['Tiempo_Prueba']:.4f} segundos")
    print("-" * 90)


C:\Users\rimel\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Tiempo de preprocesamiento total: 15.621646881103516 segundos
***************************************************************************************
 1. Grado Polinómico: 2, Combinación: age
   Máximo EA: 47938.23
   EA Medio: 9431.14
   R^2: 0.0257
   Tiempo de Generación del Modelo: 0.0050 segundos
   Tiempo de Prueba del Modelo: 0.0045 segundos
------------------------------------------------------------------------------------------
 2. Grado Polinómico: 2, Combinación: bmi
   Máximo EA: 45857.79
   EA Medio: 8947.34
   R^2: 0.0465
   Tiempo de Generación del Modelo: 0.0020 segundos
   Tiempo de Prueba del Modelo: 0.0040 segundos
------------------------------------------------------------------------------------------
 3. Grado Polinómico: 2, Combinación: region_northeast
   Máximo EA: 47468.10
   EA Medio: 8972.37
   R^2: -0.0132
   Tiempo de Generación del Modelo: 0.0040 segundos
   Tiempo de Prueba del Modelo: 0.0055 segundos
---------------------------------------------------